In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from datetime import datetime

In [2]:
class Reparametrize(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var / 2) + mean


In [3]:
hidden_size = 2

## Encoder
inputs = keras.layers.Input(shape=[69])
z = keras.layers.Dense(128)(inputs)
z = keras.layers.LeakyReLU()(z)

mean = keras.layers.Dense(hidden_size)(z)
log_var = keras.layers.Dense(hidden_size)(z)
latent = Reparametrize()([mean, log_var])

encoder = keras.Model(inputs=[inputs], outputs=[mean, log_var, latent])

## Decoder 
decoder_inputs = keras.layers.Input(shape=[hidden_size])
z = keras.layers.Dense(128)(decoder_inputs)
z = keras.layers.LeakyReLU()(z)
z = keras.layers.Dense(69)(decoder_inputs)
outputs = keras.layers.LeakyReLU()(z)

decoder = keras.Model(inputs=[decoder_inputs], outputs=[outputs])

## Joining everything
_, _, latent = encoder(inputs)
reconstruction = decoder(latent)
model = keras.Model(inputs=[inputs], outputs=[reconstruction])

In [7]:
## Loss
latent_loss = -0.5 * K.sum(1 + log_var - -K.exp(log_var) - K.square(mean), axis=-1)
model.add_loss(K.mean(latent_loss)/69)

## Compile
# The model will calculate the reconstruction loss with binary_crossentropy and then sum the KL loss to it
model.compile(loss='binary_crossentropy', optimizer="adam")
